# Apply Homography to Labels

This script applies homography to the labels from anesthesia data flowsheets and maps the bounding boxes into the unified space.

In [1]:
import sys
import os
sys.path.append(os.path.join("..", "..", "..", "ChartExtractor", "src"))

In [2]:
import json
import random
from PIL import Image, ImageDraw
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm
# Created a folder utils in the conversion folder and moved these files into there so we can call their functions
# There should be a better way to do this perhaps, if this is something we will use across various microservices
# Perhaps they can be a part of a package.
from utils.annotations import BoundingBox, Point
from utils.image_conversion import pil_to_cv2, cv2_to_pil


import cv2
import numpy as np
import pandas as pd

In [3]:
from operator import attrgetter

In [4]:
Point.__repr__ = lambda self: f"Point({self.x}, {self.y})"

---

## Load Data

In [5]:
def label_studio_to_bboxes(path_to_json_data: Path) -> List[BoundingBox]:
    """
    Loads data from LabelStudio's json format into BoundingBoxes.

    Args:
        path_to_json_data (Path) 
            A path to the json file containing the data.
    Returns:
        A list of BoundingBoxes.
    """
    json_data: List[Dict] = json.loads(open(str(path_to_json_data)).read())
    return {
        sheet_data['data']['image'].split("-")[-1]:[
            BoundingBox(
                category=label['value']['rectanglelabels'][0],
                left=label['value']['x']/100,
                top=label['value']['y']/100,
                right=label['value']['x']/100+label['value']['width']/100,
                bottom=label['value']['y']/100+label['value']['height']/100,
            )
            for label in sheet_data['annotations'][0]['result']
        ]
        for sheet_data in json_data
    }


data_path: Path = Path("..")/".."/"data"
landmark_location_data: Dict[str, List[BoundingBox]] = label_studio_to_bboxes(data_path/"intraop_document_landmarks.json")
checkbox_location_data: Dict[str, List[BoundingBox]] = label_studio_to_bboxes(data_path/"intraop_checkbox_names.json")

This is a slightly different version of the homography function from the main program. The only thing it changes is to return the homography matrix along with the image.

In [6]:
def homography_transform(
    src_image: Image.Image,
    src_points: List[Tuple[float, float]],
    dest_points: List[Tuple[float, float]],
    original_image_size: Tuple[float, float] = (3300, 2250),
) -> Tuple[List[List[float]], Image.Image]:
    """Performs homography transformation on an image.

    This function transforms an image (src_image) based on corresponding points
    between the source and destination images. It calculates the homography matrix
    and uses it to warp the source image to the perspective of the destination points.

    Args:
        src_image (Image.Image):
            A PIL image object representing the source image.
        src_points (List[Tuple[int, int]]):
            A list of tuples (x, y) representing points in the source image that correspond
            to points in the destination image.
        dest_points (List[Tuple[int, int]]):
            A list of tuples (x, y) representing points in the destination image that points
            in the source image correspond to (where the source image should be warped to).
        original_image_size (Tuple[float, float]):
            A tuple (width, height) representing the size of the control image.
            Defaults to (3300, 2250).

    Returns:
        A PIL image object representing the transformed source image.

    Raises:
        ValueError:
            If the length of src_points and dest_points don't match (must have the same
            number of corresponding points), or if there are less than 4 points.
    """
    src_points: np.ndarray = np.array(src_points)
    dest_points: np.ndarray = np.array(dest_points)

    if len(src_points) != len(dest_points):
        raise ValueError(
            "Source and destination points must have the same number of elements."
        )
    if len(src_points) < 4 or len(dest_points) < 4:
        raise ValueError("Must have 4 or more points to compute the homography.")

    src_image = pil_to_cv2(src_image)
    h, status = cv2.findHomography(src_points, dest_points)

    dest_image = cv2.warpPerspective(src_image, h, original_image_size)
    return h, cv2_to_pil(dest_image)

In [7]:
def get_corresponding_points(bboxes, imsize) -> List[Tuple[float, float]]:
    """Gets and sorts the points used for the homography from all the bounding boxes that are labeled."""
    categories_to_get = ['anesthesia_start', 'lateral', 'safety_checklist', 'units']
    if not all([c in [bb.category for bb in bboxes] for c in categories_to_get]):
        raise ValueError(f"Necessary labels not found: {categories_to_get}")
    
    points = list(map(
        attrgetter('center'),
        sorted(
            list(filter(lambda bb: bb.category in categories_to_get, bboxes)), 
            key=lambda bb: bb.category
        )
    ))
    return [(p[0]*imsize[0], p[1]*imsize[1]) for p in points]

In [8]:
remap_point = lambda p, h: cv2.perspectiveTransform(np.array(p, dtype=np.float32).reshape(-1, 1, 2), h).tolist()[0][0]


def remap_bbox(
    bbox: BoundingBox, 
    homography_matrix: List[List[float]], 
    original_width:int=4032, 
    original_height:int=3024,
    new_width:int=3300,
    new_height:int=2550,
) -> BoundingBox:
    """Maps boundingboxes to a new space using the homography matrix.
    
    Given a bounding box, homography matrix, and the image sizes of the original 
    and destination (new) image, this function returns a remapped bounding box.
    """
    new_left, new_top = remap_point((bbox.left*original_width, bbox.top*original_height), homography_matrix)
    new_right, new_bottom = remap_point((bbox.right*original_width, bbox.bottom*original_height), homography_matrix)
    return BoundingBox(bbox.category, new_left/new_width, new_top/new_height, new_right/new_width, new_bottom/new_height)

# Remap all bounding boxes
remap_all_bboxes = lambda bboxes, h: [remap_bbox(bb, h) for bb in bboxes]

### Functions To Quickly Grab the Remapped Bounding Boxes In YOLO Format

In [9]:
"""
Functions added that allow for the completion of the homography transformation and the bounding boxes for the sheet.
To be called from a for loop to complete for all documents, then exported to YOLO format.
"""

def __get_image_size(path: Path) -> Tuple[int, int]:
    """
    Returns the size of the image based on path

    Args:
        path (Path): The path to the image file
    Returns:
        Tuple[int, int]: The width and height of the image
    """
    return Image.open(data_path/"unified_intraoperative_preoperative_flowsheet_v1_1_front.png").size

# Function to convert bounding boxes to YOLO format
def convert_to_yolo_format(bbox_list):
    yolo_format = []
    for bbox in bbox_list:
        x_center = (bbox.left + bbox.right) / 2
        y_center = (bbox.top + bbox.bottom) / 2
        width = bbox.right - bbox.left
        height = bbox.bottom - bbox.top
        yolo_format.append(f"{bbox.category} {x_center} {y_center} {width} {height}")
    return yolo_format


def complete_homography_and_get_bounding_boxes(
    path_to_sheet: Path, 
    path_to_landmarks: Path,
    path_to_registered: Path,
    intraoperative: bool = True,
    show_images: bool = False,
) -> Optional[str]:
    """
    Function that completes the homography transformation and returns the bounding boxes for the sheet.

    Args:
        path_to_sheet (Path): The path to the sheet image
        path_to_landmarks (Path): The path to the landmark json file
        path_to_registered (Path): The path to the registered image directory
        intraoperative (bool, optional): Whether the sheet is intraoperative or not. Defaults to True.
        show_images (bool, optional): Whether to show the images or not. Defaults to False
    Returns:
        Optional[str]: The bounding boxes for the sheet in YOLO format. If None then the image could not be opened.
    """
    # Get the landmark location data
    data_path: Path = Path("..")/".."/"data"
    landmark_location_data: Dict[str, List[BoundingBox]] = label_studio_to_bboxes(path_to_landmarks)
    # Check if the sheet is in the landmark data
    if path_to_sheet.name not in landmark_location_data:
        print(f"Sheet {path_to_sheet.name} not found in landmark data.")
        return None
    # Get locations of landmarks for this sheet by getting the file name from the path
    locations = landmark_location_data[path_to_sheet.name]
    
    # Get the unified front/back image
    unified_file = f"unified_intraoperative_preoperative_flowsheet_v1_1_{'front' if intraoperative else 'back'}.png"
    locations_unified = landmark_location_data[unified_file]
    unified_width, unified_height = __get_image_size(data_path/unified_file)

    # Get the image
    try:
        image = Image.open(path_to_sheet)
    except:
        print(f"Unable to obtain image for sheet {path_to_sheet}. Likely in main directory and png format.")
        return None

    # Get image dimensions
    width, height = image.size

    # If show_images is true show image
    if show_images:
        image.show()

    # Perform homography transformation
    h, pil_img = homography_transform(
        src_image=image,
        src_points=get_corresponding_points(locations, (width, height)),
        dest_points=get_corresponding_points(locations_unified, (unified_width, unified_height)),
        original_image_size=(unified_width, unified_height),
    )

    # Use the homography matrix to remap all bounding boxes
    remapped_locations = remap_all_bboxes(locations, h)

    # If show_images is true show image
    if show_images:
        pil_img.show()

    # Draw bounding boxes on the image
    generate_color = lambda: "#%06x" % random.randint(0, 0xFFFFFF)
    draw = ImageDraw.Draw(pil_img)

    for bounding_box in remapped_locations:
        box = [
            bounding_box.left*unified_width,
            bounding_box.top*unified_height,
            bounding_box.right*unified_width,
            bounding_box.bottom*unified_height,
        ]
        draw.rectangle(box, outline=generate_color(), width=3)
    pil_img.resize((800, 600))

    # If show_images is true show image
    if show_images:
        pil_img.show()
        
    # Save the image
    pil_img.save(path_to_registered/path_to_sheet.name)

    return remapped_locations

### Iterate Over All Sheets, Get Bounding Boxes in YOLO For Registered Images

For each sheet, get the bounding box data in YOLO format. Make sure to create the `registered_images` directory.

In [11]:
yolo_dict = {}
for sheet in landmark_location_data:
    bounding_boxes = complete_homography_and_get_bounding_boxes(
        data_path/f"chart_images/{sheet}", 
        data_path/"intraop_document_landmarks.json", 
        data_path/"registered_images",
        show_images=False,
    )
    if bounding_boxes is None:
        continue
    yolo_boxes = convert_to_yolo_format(bounding_boxes)
    yolo_dict[sheet] = yolo_boxes

    break

print(yolo_dict)
# Save the yolo_dict to a json file
with open(data_path/"yolo_data.json", "w") as f:
    json.dump(yolo_dict, f, indent=4)

Unable to obtain image for sheet ..\..\data\chart_images\unified_intraoperative_preoperative_flowsheet_v1_1_front.png. Likely in main directory and png format.
{'RC_0001_intraoperative.JPG': ['5 0.9098491136955492 0.38141891180300247 0.0047951438210227515 0.010082098268995088', 'mg 0.9584463038589015 0.06242681765088848 0.0123184481534091 0.010029871323529414', 'mg 0.9582821377840909 0.08585675108666513 0.01217921401515154 0.009911145976945465', 'micro_g 0.9580625961766098 0.1094433474073223 0.01003543738162882 0.010558986289828431', 'pcnt 0.9571751450047348 0.7134653128829657 0.007717803030303005 0.009426604626225465', 'mmHg 0.9573366477272727 0.7376209214154412 0.02650213068181817 0.012363281249999969', 'pcnt 0.9574410363399621 0.7589949544270833 0.007789861505681839 0.009200463388480351', 'degree_C 0.9574003832267992 0.7818501311657475 0.008164284446022685 0.009822926240808827', 'ml 0.9580979965672349 0.8047243365119485 0.009312411221590877 0.00949319278492644', 'BPM 0.9578602183948